Hyper-dimensional computing on the CIFAR-10 image dataset for image classification tasks.

Get the libaries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

Load the CIFAR-10 dataset

In [ ]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Convert the images to grayscale and scale the pixel values to [0, 1]

In [ ]:
x_train_gray = np.mean(x_train, axis=-1) / 255.0
x_test_gray = np.mean(x_test, axis=-1) / 255.0

Flatten the images to 1D arrays

In [ ]:
x_train_flat = x_train_gray.reshape((x_train_gray.shape[0], -1))
x_test_flat = x_test_gray.reshape((x_test_gray.shape[0], -1))

Convert the labels to one-hot encoded vectors

In [ ]:
y_train_onehot = to_categorical(y_train)
y_test_onehot = to_categorical(y_test)

Perform PCA to reduce the dimensionality of the data to 50 dimensions

In [ ]:
pca = PCA(n_components=50)
x_train_pca = pca.fit_transform(x_train_flat)
x_test_pca = pca.transform(x_test_flat)

Define a function to compute the hypercube representation of a vector

In [ ]:
def hypercube_representation(x, d=10):
    """Compute the hypercube representation of a vector x."""
    x_hyper = np.zeros(d)
    for i in range(d):
        x_hyper[i] = np.sin((i+1) * np.pi * x.sum() / x.shape[0])
    return x_hyper

Convert the data to hypercube representation

In [ ]:

x_train_hyper = np.apply_along_axis(hypercube_representation, axis=1, arr=x_train_pca)
x_test_hyper = np.apply_along_axis(hypercube_representation, axis=1, arr=x_test_pca)

Use cross-validation to find the optimal value of k

In [ ]:
scores = []
for k in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=k)
    score = np.mean(cross_val_score(knn, x_train_hyper, y_train, cv=5))
    scores.append(score)

Print the optimal value of k and its corresponding accuracy on the test set

In [ ]:

best_k = np.argmax(scores) + 1
best_knn = KNeighborsClassifier(n_neighbors=best_k)
best_knn.fit(x_train_hyper, y_train)
accuracy = best_knn.score(x_test_hyper, y_test)

print(f"Optimal value of k: {best_k}")
print(f"Accuracy on test set: {accuracy:.4f}")